# 02 · Descriptive Statistics

Generates descriptive tables and LaTeX-ready outputs for the manuscript (Table 1). Also exports figures (boxplots/bar charts) into `../figures/`.


In [ ]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt  # Use matplotlib only (per project rules)

DATA_DERIVED = Path('../data/derived')
FIG_DIR = Path('../figures')
TAB_DIR = Path('../tables')
FIG_DIR.mkdir(parents=True, exist_ok=True)
TAB_DIR.mkdir(parents=True, exist_ok=True)


## Load clean dataset

In [ ]:
clean_path = DATA_DERIVED / 'clean_data.csv'
df = pd.read_csv(clean_path)
df['condition_label'] = df['condition_label'].astype('category')
display(df.head())

## Summary table (by condition)
Computes mean and SD for the primary DVs: post-task trust, decision time, and acceptance rate.

In [ ]:
def mean_sd(series):
    return pd.Series({'M': series.mean().round(2), 'SD': series.std(ddof=1).round(2)})

descriptives = (df
    .groupby('condition_label')
    .agg(trust_post_stats=('trust_post', mean_sd),
         decision_time_stats=('mean_decision_time_s', mean_sd),
         acceptance_rate_stats=('acceptance_rate', mean_sd)))

# Flatten multiindex
descriptives = pd.concat({
    'Trust (post, 1–5)': descriptives['trust_post_stats'],
    'Decision time (s)': descriptives['decision_time_stats'],
    'Acceptance rate (%)': descriptives['acceptance_rate_stats'],
}, axis=1)

display(descriptives)

## Export LaTeX table

In [ ]:
latex_table = descriptives.to_latex(escape=False, bold_rows=True)
with open(TAB_DIR / 'descriptives.tex', 'w') as f:
    f.write(latex_table)
print(f'Saved LaTeX table to { (TAB_DIR / 'descriptives.tex').resolve() }')

## Figures
Create manuscript figures using **matplotlib** only. Do not set custom colors; one chart per figure.

In [ ]:
# Boxplot: Trust (post)
plt.figure()
df.boxplot(column='trust_post', by='condition_label')
plt.title('Post-task Trust by Condition')
plt.suptitle('')
plt.xlabel('Condition')
plt.ylabel('Trust (1–5)')
fig_path = FIG_DIR / 'trust_boxplot.png'
plt.savefig(fig_path, dpi=300, bbox_inches='tight')
plt.close()
print(f'Saved figure: {fig_path.resolve()}')

In [ ]:
# Bar chart: Mean decision time
plt.figure()
means = df.groupby('condition_label')['mean_decision_time_s'].mean()
means.plot(kind='bar')
plt.title('Mean Decision Time by Condition')
plt.xlabel('Condition')
plt.ylabel('Decision time (s)')
fig_path = FIG_DIR / 'decision_time.png'
plt.savefig(fig_path, dpi=300, bbox_inches='tight')
plt.close()
print(f'Saved figure: {fig_path.resolve()}')

In [ ]:
# Bar chart: Acceptance rate
plt.figure()
acc_means = df.groupby('condition_label')['acceptance_rate'].mean()
acc_means.plot(kind='bar')
plt.title('Acceptance Rate by Condition')
plt.xlabel('Condition')
plt.ylabel('Acceptance rate (%))')
fig_path = FIG_DIR / 'acceptance_rate.png'
plt.savefig(fig_path, dpi=300, bbox_inches='tight')
plt.close()
print(f'Saved figure: {fig_path.resolve()}')

> Next: run `03_inferential_tests.ipynb` for assumptions and hypothesis testing.